In [ ]:
import refinitiv.data as rd

# Set up App Key
APP_KEY = "74859fbc21ff4d5aa1aa2ce565b1a8c8c0f4d37a"

# Open Refinitiv session
rd.open_session(app_key=APP_KEY)
print("Connection successful! You are now connected to the Refinitiv Data Platform.")

def list_available_fields(ticker):
    """Lists all available fields for a given ticker."""
    try:
        response = rd.content.fundamental_and_reference.Definition(universe=[ticker]).get_data()
        if response and not response.data.df.empty:
            print(f"Available fields for {ticker}:")
            for column in response.data.df.columns:
                print(column)
        else:
            print(f"No data available for {ticker}.")
    except Exception as e:
        print(f"Error fetching fields for {ticker}: {e}")

# Example: List fields for EQNR.OL
ticker = "EQNR.OL"
list_available_fields(ticker)

# Close Refinitiv session
rd.close_session()


## Felles funksjoner

In [3]:
import refinitiv.data as rd
import pandas as pd
from datetime import datetime
import os

# Set up the App Key as a shared variable
APP_KEY = "74859fbc21ff4d5aa1aa2ce565b1a8c8c0f4d37a"

# Define available fields in the shared code
available_fields = {
    1: ("TR.CompanyMarketCap", "Company Market Cap"),       # Market Cap
    2: ("TR.Revenue", "Revenue"),                          # Revenue
    3: ("TR.TotalEquity", "Total Equity"),                 # Book Equity
    4: ("TR.Beta", "Beta"),                                # Beta
    5: ("TR.CreditRatingScore", "Credit Rating"),          # Credit Rating
    6: ("TR.DebtToEquityRatio", "Debt/Equity Ratio"),      # Debt/Equity Ratio
    7: ("TR.NetDebtToEBITDA", "Net Debt/EBITDA"),          # Net Debt/EBITDA
    8: ("TR.RevenueGrowth", "Revenue Growth"),             # Revenue Growth
    9: ("TR.EBITDAGrowth", "EBITDA Growth"),               # EBITDA Growth
    10: ("TR.EBITGrowth", "EBIT Growth"),                  # EBIT Growth
    11: ("TR.NetIncomeGrowth", "Net Income Growth"),       # Net Income Growth
    12: ("TR.ReturnOnEquity", "Return on Equity"),         # Return on Equity (ROE)
    13: ("TR.ReturnOnInvestedCapital", "ROIC"),            # Return on Invested Capital (ROIC)
    14: ("TR.DividendYield", "Dividend Yield"),            # Dividend Yield
    15: ("TR.EVToEBIT", "EV/EBIT"),                        # EV/EBIT
    16: ("TR.IndustrySector", "Industry/Sector"),          # Industry/Sector
    17: ("ESG.CombinedScore", "ESG Combined Score"),       # ESG Combined Score
}

def open_refinitiv_session():
    """Opens a Refinitiv session with the Refinitiv Data Platform."""
    rd.open_session(app_key=APP_KEY)
    print("Connection successful! You are now connected to the Refinitiv Data Platform.")

def fetch_data(tickers, selected_field, expected_column, currency):
    """Fetches data from Refinitiv based on tickers, field, and expected column."""
    data = pd.DataFrame()  # Collects data for all companies

    for ticker in tickers:
        try:
            response = rd.content.fundamental_and_reference.Definition(
                universe=[ticker],
                fields=[selected_field],
                parameters={"Scale": 6, "SDate": "1990-01-01", "EDate": -1, "FRQ": "FY", "Curn": currency}
            ).get_data()

            if response and not response.data.df.empty:
                df = response.data.df

                if expected_column not in df.columns:
                    print(f"Column '{expected_column}' does not exist in the response for {ticker}. Skipping.")
                    continue

                company_df = df[[expected_column]].copy()
                company_df.columns = [ticker]

                available_dates = df.get("Date")
                if available_dates is not None:
                    company_df["Date"] = pd.to_datetime(available_dates)
                else:
                    num_rows = company_df.shape[0]
                    current_year = datetime.now().year
                    dates = pd.date_range(end=f"{current_year-2}-12-31", periods=num_rows, freq="YE")
                    company_df["Date"] = dates

                company_df = company_df[company_df["Date"] <= pd.to_datetime(f"{datetime.now().year}-12-31")]
                company_df.set_index("Date", inplace=True)

                data = pd.concat([data, company_df], axis=1)
                print(f"Data retrieved for {ticker}")
            else:
                print(f"No data available for {ticker}")

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data

def save_data_to_excel(retrieved_data, file_name, sheet_name):
    """Saves data to an Excel file with a specified file name and sheet name."""
    if not retrieved_data.empty:
        # Ensure the directory exists
        folder_name = os.path.dirname(file_name)
        if folder_name and not os.path.exists(folder_name):
            os.makedirs(folder_name, exist_ok=True)

        # Save or write to the Excel file
        with pd.ExcelWriter(file_name, mode="w", engine="openpyxl") as writer:
            retrieved_data.to_excel(writer, sheet_name=sheet_name)
        print(f"Data saved in '{file_name}', Sheet: {sheet_name}.")
    else:
        print("No data available to save.")

def close_refinitiv_session():
    """Closes the Refinitiv session."""
    rd.close_session()
    print("Session closed.")


## Test

In [1]:
# Open Refinitiv session
open_refinitiv_session()

# Ask the user what they want to fetch
print("What do you want to fetch? Select one field from the list:")
for key, value in available_fields.items():
    print(f"{key}: {value[1]}")

try:
    selection = int(input("Your selection: "))
    if selection not in available_fields:
        print("Invalid selection. Program will exit.")
        close_refinitiv_session()
        exit()
    selected_field, expected_column = available_fields[selection]
except Exception as e:
    print(f"Error with selection: {e}. Program will exit.")
    close_refinitiv_session()
    exit()

# Configuration for Norway
norwegian_tickers = ["EQNR.OL", "NHY.OL", "TEL.OL", "YAR.OL"]  # Norwegian tickers
currency = "NOK"  # Currency for Norway
file_name = f"Factors/{expected_column}.xlsx"  # File name based on field

# Fetch data
retrieved_data = fetch_data(norwegian_tickers, selected_field, expected_column, currency)

# Save data
save_data_to_excel(retrieved_data, file_name, sheet_name="Norway")

# Close Refinitiv session
close_refinitiv_session()


NameError: name 'open_refinitiv_session' is not defined

## Hente ut Norske data

In [75]:
import refinitiv.data as rd
import pandas as pd
from datetime import datetime
import os
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)  # Undertrykk FutureWarning

# Sett opp Refinitiv-session med App Key
APP_KEY = "74859fbc21ff4d5aa1aa2ce565b1a8c8c0f4d37a"
rd.open_session(app_key=APP_KEY)
print("Tilkobling vellykket! Du er nå koblet til Refinitiv Data Platform.")

# Tilgjengelige felt med mapping til kolonnenavn
tilgjengelige_felter = {
    1: ("TR.CompanyMarketCap", "Company Market Cap"),  # Market Cap
    2: ("TR.Revenue", "Revenue"),                     # Revenue
}

# Funksjon for å hente data basert på valgt felt
def hent_data(tickers, valgt_felt_key):
    valgt_felt, forventet_kolonne = tilgjengelige_felter[valgt_felt_key]
    data = pd.DataFrame()  # Samler data for alle selskaper

    for ticker in tickers:
        try:
            response = rd.content.fundamental_and_reference.Definition(
                universe=[ticker],
                fields=[valgt_felt],
                parameters={"Scale": 6, "SDate": "1990-01-01", "EDate": -1, "FRQ": "FY", "Curn": "NOK"}
            ).get_data()

            if response and not response.data.df.empty:
                df = response.data.df

                # Sjekk om den forventede kolonnen finnes
                if forventet_kolonne not in df.columns:
                    print(f"Kolonnen '{forventet_kolonne}' finnes ikke i responsen for {ticker}. Hopper over.")
                    continue

                # Hent ønsket data og legg til dato
                selskap_df = df[[forventet_kolonne]].copy()
                selskap_df.columns = [ticker]

                tilgjengelige_datoer = df.get("Date")
                if tilgjengelige_datoer is not None:
                    selskap_df["Dato"] = pd.to_datetime(tilgjengelige_datoer)
                else:
                    antall_rader = selskap_df.shape[0]
                    nåværende_år = datetime.now().year
                    datoer = pd.date_range(end=f"{nåværende_år-2}-12-31", periods=antall_rader, freq="YE")
                    selskap_df["Dato"] = datoer
                
                selskap_df = selskap_df[selskap_df["Dato"] <= pd.to_datetime(f"{datetime.now().year}-12-31")]
                selskap_df.set_index("Dato", inplace=True)

                data = pd.concat([data, selskap_df], axis=1)
                print(f"Data hentet for {ticker}")
            else:
                print(f"Ingen data tilgjengelig for {ticker}")

        except Exception as e:
            print(f"Feil ved henting av {ticker}: {e}")

    return data

# Liste over norske aksjer
norske_aksjer = ["EQNR.OL", "NHY.OL", "TEL.OL", "YAR.OL"]  # Ekskluder DNB midlertidig

# Spør brukeren hva de vil hente
print("Hva vil du hente? Velg et nummer fra listen:")
for key, value in tilgjengelige_felter.items():
    print(f"{key}: {value[1]}")  # Vis kun beskrivelsen (forventet kolonnenavn)

try:
    valg = int(input("Ditt valg: "))  # Brukeren skriver et nummer
    if valg not in tilgjengelige_felter:
        print("Ugyldig valg. Programmet avsluttes.")
        rd.close_session()
        exit()
except ValueError:
    print("Ugyldig input. Programmet avsluttes.")
    rd.close_session()
    exit()

# Hent data basert på brukerens valg
hentet_data = hent_data(norske_aksjer, valg)

# Dynamisk lagring i Excel basert på valg
_, kolonnenavn = tilgjengelige_felter[valg]
mappenavn = "Factors"
filnavn = os.path.join(mappenavn, f"{kolonnenavn}.xlsx")

# Lagre data i Excel hvis noe ble hentet
if not hentet_data.empty:
    # Opprett mappen hvis den ikke eksisterer
    os.makedirs(mappenavn, exist_ok=True)

    # Lag eller skriv til Excel-fil
    if not os.path.exists(filnavn):
        mode = "w"  # Lag en ny fil hvis den ikke eksisterer
    else:
        mode = "a"  # Append til eksisterende fil

    with pd.ExcelWriter(filnavn, mode=mode, engine="openpyxl", if_sheet_exists="replace") as writer:
        hentet_data.to_excel(writer, sheet_name="Norge")
    print(f"Data lagret i '{filnavn}', Sheet: Norge.")
else:
    print("Ingen data tilgjengelig å lagre.")

# Lukk session
rd.close_session()


Tilkobling vellykket! Du er nå koblet til Refinitiv Data Platform.
Hva vil du hente? Velg et nummer fra listen:
1: Company Market Cap
2: Revenue
Data hentet for EQNR.OL
Data hentet for NHY.OL
Data hentet for TEL.OL
Data hentet for YAR.OL
Data lagret i 'Factors\Revenue.xlsx', Sheet: Norge.


## Sverige


In [63]:
# Åpne Refinitiv-session
open_refinitiv_session()

# Tilgjengelige felt
tilgjengelige_felter = {
    1: ("TR.CompanyMarketCap", "Company Market Cap"),  # Market Cap
    2: ("TR.Revenue", "Revenue"),                     # Revenue
}

# Spør brukeren hva de vil hente
print("Hva vil du hente? Velg et nummer fra listen:")
for key, value in tilgjengelige_felter.items():
    print(f"{key}: {value[1]}")  # Vis beskrivelsen av feltet

try:
    valg = int(input("Ditt valg: "))
    if valg not in tilgjengelige_felter:
        print("Ugyldig valg. Programmet avsluttes.")
        close_refinitiv_session()
        exit()
except ValueError:
    print("Ugyldig input. Programmet avsluttes.")
    close_refinitiv_session()
    exit()

# Hent konfigurasjon for Sverige
valgt_felt, forventet_kolonne = tilgjengelige_felter[valg]
svenske_aksjer = ["ERICb.ST", "VOLVb.ST", "ATCOb.ST", "SAND.ST", "HMb.ST"]  # Justert aksjeliste med korrekte tickere
filnavn = f"Factors/{forventet_kolonne}.xlsx"  # Dynamisk filnavn basert på feltet
sheet_name = "Sverige"  # Ark-navn i Excel

# Hent data
hentet_data = hent_data(svenske_aksjer, valgt_felt, forventet_kolonne, valuta="SEK")

# Lagre data i Excel
lagre_data_i_excel(hentet_data, filnavn, sheet_name)

# Lukk Refinitiv-session
close_refinitiv_session()



Tilkobling vellykket! Du er nå koblet til Refinitiv Data Platform.
Hva vil du hente? Velg et nummer fra listen:
1: Company Market Cap
2: Revenue
Data hentet for ERICb.ST
Data hentet for VOLVb.ST
Data hentet for ATCOb.ST
Data hentet for SAND.ST
Data hentet for HMb.ST
Data lagret i 'Factors/Revenue.xlsx', Sheet: Sverige.
Session lukket.


## Danmark

In [65]:
# Åpne Refinitiv-session
open_refinitiv_session()

# Tilgjengelige felt
tilgjengelige_felter = {
    1: ("TR.CompanyMarketCap", "Company Market Cap"),  # Market Cap
    2: ("TR.Revenue", "Revenue"),                     # Revenue
}

# Spør brukeren hva de vil hente
print("Hva vil du hente? Velg et nummer fra listen:")
for key, value in tilgjengelige_felter.items():
    print(f"{key}: {value[1]}")  # Vis beskrivelsen av feltet

try:
    valg = int(input("Ditt valg: "))
    if valg not in tilgjengelige_felter:
        print("Ugyldig valg. Programmet avsluttes.")
        close_refinitiv_session()
        exit()
except ValueError:
    print("Ugyldig input. Programmet avsluttes.")
    close_refinitiv_session()
    exit()

# Hent konfigurasjon for Danmark
valgt_felt, forventet_kolonne = tilgjengelige_felter[valg]
danske_aksjer = ["NOVOb.CO", "CARLb.CO", "DSV.CO", "VWS.CO", "ORSTED.CO"]  # Justert aksjeliste
filnavn = f"Factors/{forventet_kolonne}.xlsx"  # Dynamisk filnavn basert på feltet
sheet_name = "Danmark"  # Ark-navn i Excel

# Hent data
hentet_data = hent_data(danske_aksjer, valgt_felt, forventet_kolonne, valuta="DKK")

# Lagre data i Excel
lagre_data_i_excel(hentet_data, filnavn, sheet_name)

# Lukk Refinitiv-session
close_refinitiv_session()


Tilkobling vellykket! Du er nå koblet til Refinitiv Data Platform.
Hva vil du hente? Velg et nummer fra listen:
1: Company Market Cap
2: Revenue
Data hentet for NOVOb.CO
Data hentet for CARLb.CO
Data hentet for DSV.CO
Data hentet for VWS.CO
Data hentet for ORSTED.CO
Data lagret i 'Factors/Revenue.xlsx', Sheet: Danmark.
Session lukket.


## Finland

In [67]:
# Åpne Refinitiv-session
open_refinitiv_session()

# Tilgjengelige felt
tilgjengelige_felter = {
    1: ("TR.CompanyMarketCap", "Company Market Cap"),  # Market Cap
    2: ("TR.Revenue", "Revenue"),                     # Revenue
}

# Spør brukeren hva de vil hente
print("Hva vil du hente? Velg et nummer fra listen:")
for key, value in tilgjengelige_felter.items():
    print(f"{key}: {value[1]}")  # Vis beskrivelsen av feltet

try:
    valg = int(input("Ditt valg: "))
    if valg not in tilgjengelige_felter:
        print("Ugyldig valg. Programmet avsluttes.")
        close_refinitiv_session()
        exit()
except ValueError:
    print("Ugyldig input. Programmet avsluttes.")
    close_refinitiv_session()
    exit()

# Hent konfigurasjon for Finland
valgt_felt, forventet_kolonne = tilgjengelige_felter[valg]
finske_aksjer = ["NOKIA.HE", "KNEBV.HE", "UPM.HE", "WRT1V.HE", "METSB.HE"]  # Justert aksjeliste
filnavn = f"Factors/{forventet_kolonne}.xlsx"  # Dynamisk filnavn basert på feltet
sheet_name = "Finland"  # Ark-navn i Excel

# Hent data
hentet_data = hent_data(finske_aksjer, valgt_felt, forventet_kolonne, valuta="EUR")

# Lagre data i Excel
lagre_data_i_excel(hentet_data, filnavn, sheet_name)

# Lukk Refinitiv-session
close_refinitiv_session()


Tilkobling vellykket! Du er nå koblet til Refinitiv Data Platform.
Hva vil du hente? Velg et nummer fra listen:
1: Company Market Cap
2: Revenue
Data hentet for NOKIA.HE
Data hentet for KNEBV.HE
Data hentet for UPM.HE
Data hentet for WRT1V.HE
Data hentet for METSB.HE
Data lagret i 'Factors/Revenue.xlsx', Sheet: Finland.
Session lukket.
